In [ ]:
import os
import numpy as np
from sklearn.metrics import f1_score
from skimage.io import imread
from skimage.transform import resize,rotate
from skimage.measure import find_contours
from skimage.util import invert
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
plt.style.use('default')
from keras.optimizers import SGD, Adam
import keras.backend as K
import keras
from keras import layers, activations
from keras.models import Model
from keras.preprocessing.image import load_img,img_to_array
from keras.utils.vis_utils import plot_model
from keras.engine.topology import Layer
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from IPython.display import Image
import tensorflow as tf
import Augmentor
import yaml
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_x = []
train_y = []
test_x = []
test_y = []

####path
images_names_train=os.listdir('dataset-5.0/train/image/')
mask_names_train=os.listdir('dataset-5.0/train/annotation/')
############################################################
images_names_test=os.listdir('dataset-4.0/test/image/')
mask_names_test=os.listdir('dataset-4.0/test/annotation/')
####dataset construct
for i in images_names_train:
    x_path = os.path.join('dataset-5.0/train/image/'+ i) 
    img = load_img(x_path,target_size=(128,128))
    img = img_to_array(img)/255
    train_x.append(img)
for j in mask_names_train:
    y_path = os.path.join('dataset-5.0/train/annotation/'+ j) 
    img = load_img(y_path,target_size=(128,128))
    img = img_to_array(img)/255
    
    weed = img[:,:,0]
    crop = img[:,:,1]

    y = np.zeros((128,128,3))
    y[:,:, 0] = weed
    y[:, :, 1] = crop
    
    back = img.copy()
    back = np.all(img == [0, 0, 0],axis=-1)*1
    y[:,:, 2] = back
    
    y = np.reshape(y,(128*128,3))
    
    train_y.append(y)
############################################################
for i in images_names_test:
    x_path = os.path.join('dataset-4.0/test/image/'+ i) 
    img = load_img(x_path,target_size=(128,128))
    img = img_to_array(img)/255
    test_x.append(img)
for j in mask_names_test:
    y_path = os.path.join('dataset-4.0/test/annotation/'+ j) 
    img = load_img(y_path,target_size=(128,128))
    img = img_to_array(img)/255
    
    weed = img[:,:,0]
    crop = img[:,:,1]

    y = np.zeros((128,128,3))
    y[:,:, 0] = weed
    y[:, :, 1] = crop
    
    back = img.copy()
    back = np.all(img == [0, 0, 0],axis=-1)*1
    y[:,:, 2] = back
    
    y = np.reshape(y,(128*128,3))
    
    test_y.append(y)

train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

#analise pixel lvl for classess
weed_pixel = 0
crop_pixel = 0
background_pixel = 0
for j in mask_names_train:
    y_path = os.path.join('dataset-4.0/train/annotation/'+ j) 
    img = load_img(y_path,target_size=(128,128))
    img = img_to_array(img)/255
    weed = img[:,:,0]
    _pixel = np.sum(weed.flatten())
    weed_pixel += _pixel 
    crop = img[:,:,1]
    _pixel = np.sum(crop.flatten())
    crop_pixel += _pixel

    back = img.copy()
    back = np.all(img == [0, 0, 0],axis=-1) * 1
    _pixel = np.sum(back.flatten())
    background_pixel += _pixel

categorical_weight = [background_pixel/weed_pixel, background_pixel/crop_pixel, 1]
def weights_categorical_crossentropy(weights):
   
    if isinstance(weights,list) or isinstance(np.ndarray):
        weights=K.variable(weights)

    def loss(target,output,from_logits=False):
        if not from_logits:
            output /= tf.reduce_sum(output,
                                    len(output.get_shape()) - 1,
                                    True)
            _epsilon = tf.convert_to_tensor(K.epsilon(), dtype=output.dtype.base_dtype)
            output = tf.clip_by_value(output, _epsilon, 1. - _epsilon)
            weighted_losses = target * tf.log(output) * weights
            return - tf.reduce_sum(weighted_losses,len(output.get_shape()) - 1)
        else:
            raise ValueError('error')
    return loss


def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)
def precision(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    return K.mean(p)

def recall(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    return K.mean(r)
    

In [ ]:

json_file = open('model_b.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.compile(loss=custom_loss,optimizer=adam,metrics=['accuracy',f1,precision,recall,mean_iou])
# load weights into new model
loaded_model.load_weights("model_b.h5")
print("Loaded model from disk")